# 일정 기간 (2년)내 채무불이행 여부 파악!

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 데이터 불러오기 및 전처리

In [3]:
train = pd.read_csv('data/cs-training.csv', index_col = 0)
train

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149996,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149997,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149998,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149999,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


### 컬럼명 단순화

In [4]:
train.columns = ['more90_2yrs', 'rev_util', 'age', 'num_30-59_due', 'debt_ratio', 'month_income', 'num_cre_loan', 'num_90late', 'num_REL', 'num_60-89_due', 'num_dependents']
train

,more90_2yrs,rev_util,age,num_30-59_due,debt_ratio,month_income,num_cre_loan,num_90late,num_REL,num_60-89_due,num_dependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149996,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149997,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149998,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149999,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


In [5]:
train.head(10)

,more90_2yrs,rev_util,age,num_30-59_due,debt_ratio,month_income,num_cre_loan,num_90late,num_REL,num_60-89_due,num_dependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,NaN,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,NaN,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0


In [6]:
train.tail(10)

,more90_2yrs,rev_util,age,num_30-59_due,debt_ratio,month_income,num_cre_loan,num_90late,num_REL,num_60-89_due,num_dependents
149991,0,0.055518,46,0,0.609779,4335.0,7,0,1,0,2.0
149992,0,0.104112,59,0,0.477658,10316.0,10,0,2,0,0.0
149993,0,0.871976,50,0,4132.000000,NaN,11,0,1,0,3.0
149994,0,1.000000,22,0,0.000000,820.0,1,0,0,0,0.0
149995,0,0.385742,50,0,0.404293,3400.0,7,0,0,0,0.0
149996,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149997,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149998,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149999,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0
150000,0,0.850283,64,0,0.249908,8158.0,8,0,2,0,0.0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
more90_2yrs       150000 non-null int64
rev_util          150000 non-null float64
age               150000 non-null int64
num_30-59_due     150000 non-null int64
debt_ratio        150000 non-null float64
month_income      120269 non-null float64
num_cre_loan      150000 non-null int64
num_90late        150000 non-null int64
num_REL           150000 non-null int64
num_60-89_due     150000 non-null int64
num_dependents    146076 non-null float64
dtypes: float64(4), int64(7)
memory usage: 13.7 MB


In [8]:
train.more90_2yrs.value_counts()

0    139974
1     10026
Name: more90_2yrs, dtype: int64

In [9]:
train.age.value_counts().sort_index()

0        1
21     183
22     434
23     641
24     816
      ... 
102      3
103      3
105      1
107      1
109      2
Name: age, Length: 86, dtype: int64

### 결측치 확인

In [10]:
train.isna().sum()

more90_2yrs           0
rev_util              0
age                   0
num_30-59_due         0
debt_ratio            0
month_income      29731
num_cre_loan          0
num_90late            0
num_REL               0
num_60-89_due         0
num_dependents     3924
dtype: int64

### 결측치 제거

In [11]:
train.dropna(inplace= True)

In [12]:
train.isna().sum()

more90_2yrs       0
rev_util          0
age               0
num_30-59_due     0
debt_ratio        0
month_income      0
num_cre_loan      0
num_90late        0
num_REL           0
num_60-89_due     0
num_dependents    0
dtype: int64

In [13]:
train.size

1322959

In [57]:
train.columns

Index(['more90_2yrs', 'rev_util', 'age', 'num_30-59_due', 'debt_ratio',
       'month_income', 'num_cre_loan', 'num_90late', 'num_REL',
       'num_60-89_due', 'num_dependents'],
      dtype='object')

## 데이터셋 분리

In [15]:
from sklearn.model_selection import train_test_split
X = train.drop(columns = ['more90_2yrs'])
y = train.more90_2yrs
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [16]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((90201, 10), (30068, 10), (90201,), (30068,))

## 결정트리

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
tree = DecisionTreeClassifier(random_state = 1)

In [19]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best')

## 모델 평가

In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [21]:
# 평가지표 출력하는 함수

def print_metrics(y, pred, title = None):
    acc = accuracy_score(y, pred)
    recall = recall_score(y, pred)
    precision = precision_score(y, pred)
    f1 = f1_score(y, pred)
    
    if title:
        print(title)
    print(f'정확도: {acc}, 재현율: {recall}, 정밀도: {precision}, f1: {f1}')

In [22]:
# 훈련 데이터셋으로 예측
pred_train = tree.predict(X_train)
pred_train

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [23]:
# 테스트 데이터셋으로 예측
pred_test = tree.predict(X_test)
pred_test

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [24]:
y_test

42419     0
36051     1
112561    0
49412     0
35560     0
         ..
20887     0
57566     0
93175     0
8688      1
24382     0
Name: more90_2yrs, Length: 30068, dtype: int64

In [25]:
print("Train셋의 정확도: ", accuracy_score(y_train, pred_train))

Train셋의 정확도:  0.9999889136484075


In [26]:
print("Test셋의 정확도: ", accuracy_score(y_test, pred_test))

Test셋의 정확도:  0.8900824797126513


In [27]:
print_metrics(y_train, pred_train)

정확도: 0.9999889136484075, 재현율: 1.0, 정밀도: 0.9998404849258254, f1: 0.9999202361011406


In [28]:
print_metrics(y_test, pred_test)

정확도: 0.8900824797126513, 재현율: 0.2666347534705601, 정밀도: 0.2390557939914163, f1: 0.25209323376329484


### 랜덤포레스트 & 모델 평가

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
print_metrics(y_test, pred_rf, 'RF - test')

C:\Users\amelie\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF - test
정확도: 0.9294598909139284, 재현율: 0.16371469602680708, 정밀도: 0.4776536312849162, f1: 0.24385026737967913


### ROC, AUC 확인

In [31]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [40]:
pred_rf_proba = rf.predict_proba(X_test)

In [43]:
pred_rf_proba.shape

(30068, 2)

In [47]:
pred_rf = rf.predict(X_test)
pred_rf_proba = rf.predict_proba(X_test)

In [56]:
pred_rf_proba[:, 1]

array([0. , 0.1, 0. , ..., 0. , 0.1, 0. ])

In [54]:
auc = roc_auc_score(y_test, pred_rf_proba[:,1])

In [55]:
auc

0.7740375308151426